In [ ]:
### From NN_Regression_Velocity

In [ ]:
### Data processing section 
# Order in bigData is [tri -- squ -- pent -- hex]
processData = False

# Safety mechanism to not override data
if processData:
    testPathBase='compVal'
    NN = np.array([1, 2, 5])
    lenTraj = 6
    nFilt = 3
    nGeom = 4
    nSeed = 5
    nLoocv = 8
    BIGDAT = np.zeros((nFilt*nGeom*nSeed*nLoocv, 5))
    for ii in range(nFilt):
        for jj in np.arange(3, 3+nGeom):
            geometryVal = jj
            for ll in range(1, 1+nLoocv):
                for kk in range(nSeed):
                    if geometryVal == 3:
                        geomPath='tri/'
                        geoString = 'TRI'
                    elif geometryVal == 4:
                        geomPath='squ/'
                        geoString = 'SQU'
                    elif geometryVal == 5:
                        geomPath='pent/'
                        geoString = 'PENT'
                    elif geometryVal == 6:
                        geomPath='hex/'
                        geoString = 'HEX'
                    else:
                        raise ValueError('Geometry must be in {3, 4, 5, 6}')

                    testPath=testPathBase+'_N'+str(NN[ii])+'/' + geomPath + 'valCost'+str(kk)+'_'+str(ll)+'.csv'
                    # testPathInit = 'compVal_N'+str(NN[ii])+'/' + geomPath + 'testCostInit'+str(kk)+'.csv'

                    # tmp[0,:2] = pd.read_csv(testPathInit, header=None).to_numpy()
                    tmp0 = pd.read_csv(testPath, header=None).to_numpy()
                    tmp = np.zeros(5)
                    tmp[0] = tmp0[-1,-1]
                    # tmp[1:,0] += 1
                    tmp[1] = kk+1
                    tmp[2] = geometryVal
                    tmp[3] = NN[ii]
                    tmp[4] = ll

                    BIGDAT[(ii*nGeom*nSeed*nLoocv + (jj-3)*nSeed*nLoocv + (ll-1)*nSeed + kk), :] = tmp

    # Save to npy file
    np.save('SummaryStats/BigDataVelMax.npy', BIGDAT)
else: 
    raise ValueError('Did not process the data due to the processData flag being False. Make sure you want to process the data before proceeding!')

In [ ]:
# Process testErrs into a dataframe and condense to averages
# Order in bigData is [tri -- squ -- pent -- hex]
processData=False

# Safety mechanism to not override data
if processData:
    
    testPathBase='compVal'
    NN = np.array([1, 2, 5])
    lenDat = 89999
    nFilt = len(NN)
    nGeom = 4
    nSeed = 5
    nLoocv = 8
    dfMain = pd.DataFrame(columns=('Vel_ms', 'Err_ms', 'N', 'Geometry'))
    dfMain2 = pd.DataFrame(columns=('Vel_ms', 'Err_ms', 'N', 'Geometry'))
    for ii in range(nFilt):
        for jj in np.arange(3, 3+nGeom):
            geometryVal = jj
            if geometryVal == 3:
                geomPath='tri/'
                geoString = 'TRI'
            elif geometryVal == 4:
                geomPath='squ/'
                geoString = 'SQU'
            elif geometryVal == 5:
                geomPath='pent/'
                geoString = 'PENT'
            elif geometryVal == 6:
                geomPath='hex/'
                geoString = 'HEX'
            else:
                raise ValueError('Geometry must be in {3, 4, 5, 6}')
                
            tmpDAT = np.zeros((nSeed*nLoocv*lenDat, 5))
            bestSeed = 900.0*np.ones((nSeed, nLoocv))
            for kk in range(nSeed):
                for ll in range(nLoocv):
                    trainPath = testPathBase+'_N'+str(NN[ii])+'/' + geomPath + 'trainCost'+str(kk)+'_'+str(ll+1)+'.csv' 
                    testPath = testPathBase+'_N'+str(NN[ii])+'/' + geomPath + 'valErrs'+str(kk)+'_'+str(ll+1)+'.csv'
                    # testPathInit = 'compVal_N'+str(NN[ii])+'/' + geomPath + 'testCostInit'+str(kk)+'.csv'

                    tmp0 = pd.read_csv(trainPath, header=None).to_numpy()
                    bestSeed[kk, ll] = tmp0[-1, 1]
                    # print(tmp0[-1, 1])

                    tmp = np.zeros((lenDat, 5))
                    # tmp[0,:2] = pd.read_csv(testPathInit, header=None).to_numpy()
                    '''
                        # tmp90 = pd.read_csv(testPath, header=None).to_numpy()
                        # print(tmp90[:2,:2])
                        # print(tmp90.shape)
                        # breakpoint()
                    '''
                    tmp[:,:2] = (pd.read_csv(testPath, header=None).to_numpy()[:lenDat,:])
                    # tmp[:,:2] *= 180.0/math.pi
                    tmp[:,0] = tmp[:,0]
                    # bestSeed[kk] = np.mean(tmp[:,1])
                    tmp[:,2] = np.ones(lenDat)*NN[ii]
                    tmp[:,3] = np.ones(lenDat)*geometryVal
                    tmp[:,4] = np.ones(lenDat)*(kk+1)
                    tmpDAT[(kk*nLoocv*lenDat+ll*lenDat):(kk*nLoocv*lenDat+lenDat*(ll+1)),:] = tmp


            # Choose the best seed for each geometry-filter configuration
            BS = np.mean(bestSeed, axis=1)
            try:
                len(BS)==nSeed
            except:
                raise ValueError('Taking mean along the incorrect axis! [Line 63 of NN_Regression_Velocity]')
            
            kkStar = np.argmin(BS)
            # print(bestSeed)
            # print(f"The best seed is Seed {kkStar+1}, representing element"+ f" [{kkStar}] of bestSeed array")
            # breakpoint()

            dfTmp = pd.DataFrame(data=tmpDAT[kkStar*nLoocv*lenDat:(kkStar+1)*nLoocv*lenDat,:4],
                                 columns=('Vel_ms', 'Err_ms', 'N', 'Geometry')
                                )

            dfMain = pd.concat([dfMain, dfTmp])

            dfTmp2 = dfTmp.groupby(['Vel_ms'], as_index=False).mean()
            # print(dfTmp2.shape)
            # Append to main dataframe
            dfMain2 = pd.concat([dfMain2, dfTmp2])
            # print(dfMain.shape)


            # BIGDAT[(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + kk*lenTraj):(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + (kk+1)*lenTraj), :] = tmp

    # Save to csv file
    dfMain.to_csv('SummaryStats/VelErrorVSAngleLarge.csv')
    dfMain2.to_csv('SummaryStats/VelErrorVSAngle.csv')

else: 
    raise ValueError('Did not process the data due to the processData flag being False. Make sure you want to process the data before proceeding!')

In [ ]:
### From NN_Regression_Angles

In [ ]:
### Data processing section 
# Order in bigData is [tri -- squ -- pent -- hex]
processData = False

# Safety mechanism to not override data
if processData:
    testPathBase='compTest'
    NN = np.array([1, 2, 5])
    lenTraj = 21
    nFilt = 3
    nGeom = 4
    nSeed = 5
    BIGDAT = np.zeros((nFilt*nGeom*nSeed*lenTraj, 5))
    for ii in range(nFilt):
        for jj in np.arange(3, 3+nGeom):
            geometryVal = jj
            for kk in range(nSeed):
                if geometryVal == 3:
                    geomPath='tri/'
                    geoString = 'TRI'
                elif geometryVal == 4:
                    geomPath='squ/'
                    geoString = 'SQU'
                elif geometryVal == 5:
                    geomPath='pent/'
                    geoString = 'PENT'
                elif geometryVal == 6:
                    geomPath='hex/'
                    geoString = 'HEX'
                else:
                    raise ValueError('Geometry must be in {3, 4, 5, 6}')

                testPath=testPathBase+'_N'+str(NN[ii])+'/' + geomPath + 'testCost'+str(kk)+'.csv'
                # testPathInit = 'compVal_N'+str(NN[ii])+'/' + geomPath + 'testCostInit'+str(kk)+'.csv'

                tmp = np.zeros((lenTraj, 5))
                # tmp[0,:2] = pd.read_csv(testPathInit, header=None).to_numpy()
                tmp[0:,:2] = pd.read_csv(testPath, header=None).to_numpy()
                # tmp[1:,0] += 1
                tmp[:,2] = np.ones(lenTraj)*(kk+1)
                tmp[:,3] = np.ones(lenTraj)*geometryVal
                tmp[:,4] = np.ones(lenTraj)*NN[ii]

                BIGDAT[(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + kk*lenTraj):(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + (kk+1)*lenTraj), :] = tmp

    # Save to npy file
    np.save('SummaryStats/BigData.npy', BIGDAT)
else: 
    raise ValueError('Did not process the data due to the processData flag being False. Make sure you want to process the data before proceeding!')

In [ ]:
# Process testErrs into a dataframe and condense to averages
# Order in bigData is [tri -- squ -- pent -- hex]

# Safety mechanism to not override data
if processData:
    
    testPathBase='compTest'
    NN = np.array([1, 2, 5])
    lenDat = 287999
    nFilt = len(NN)
    nGeom = 4
    nSeed = 5
    dfMain = pd.DataFrame(columns=('Angle_deg', 'Err_deg', 'N', 'Geometry'))
    dfMain2 = pd.DataFrame(columns=('Angle_deg', 'Err_deg', 'N', 'Geometry'))
    for ii in range(nFilt):
        for jj in np.arange(3, 3+nGeom):
            geometryVal = jj
            tmpDAT = np.zeros((nSeed*lenDat, 5))
            bestSeed = 900.0*np.ones(nSeed)
            for kk in range(nSeed):
                if geometryVal == 3:
                    geomPath='tri/'
                    geoString = 'TRI'
                elif geometryVal == 4:
                    geomPath='squ/'
                    geoString = 'SQU'
                elif geometryVal == 5:
                    geomPath='pent/'
                    geoString = 'PENT'
                elif geometryVal == 6:
                    geomPath='hex/'
                    geoString = 'HEX'
                else:
                    raise ValueError('Geometry must be in {3, 4, 5, 6}')

                trainPath = testPathBase+'_N'+str(NN[ii])+'/' + geomPath + 'trainCost'+str(kk)+'.csv' 
                testPath = testPathBase+'_N'+str(NN[ii])+'/' + geomPath + 'testErrs'+str(kk)+'.csv'
                # testPathInit = 'compVal_N'+str(NN[ii])+'/' + geomPath + 'testCostInit'+str(kk)+'.csv'

                tmp0 = pd.read_csv(trainPath, header=None).to_numpy()
                bestSeed[kk] = tmp0[-1, 1]
                # print(tmp0[-1, 1])

                tmp = np.zeros((lenDat, 5))
                # tmp[0,:2] = pd.read_csv(testPathInit, header=None).to_numpy()
                tmp[:,:2] = pd.read_csv(testPath, header=None).to_numpy()
                tmp[:,:2] *= 180.0/math.pi
                tmp[:,0] = np.round_(tmp[:,0])
                # bestSeed[kk] = np.mean(tmp[:,1])
                tmp[:,2] = np.ones(lenDat)*NN[ii]
                tmp[:,3] = np.ones(lenDat)*geometryVal
                tmp[:,4] = np.ones(lenDat)*(kk+1)
                tmpDAT[kk*lenDat:(kk+1)*lenDat,:] = tmp


            # Choose the best seed for each geometry-filter configuration
            kkStar = np.argmin(bestSeed)
            # print(bestSeed)
            # print(f"The best seed is Seed {kkStar+1}, representing element"+ f" [{kkStar}] of bestSeed array")
            # breakpoint()

            dfTmp = pd.DataFrame(data=tmpDAT[kkStar*lenDat:(kkStar+1)*lenDat,:4],
                                 columns=('Angle_deg', 'Err_deg', 'N', 'Geometry')
                                )

            dfMain = pd.concat([dfMain, dfTmp])

            dfTmp2 = dfTmp.groupby(['Angle_deg'], as_index=False).mean()
            # print(dfTmp2.shape)
            # Append to main dataframe
            dfMain2 = pd.concat([dfMain2, dfTmp2])
            # print(dfMain.shape)


            # BIGDAT[(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + kk*lenTraj):(ii*nGeom*nSeed*lenTraj + (jj-3)*nSeed*lenTraj + (kk+1)*lenTraj), :] = tmp

    # Save to csv file
    dfMain.to_csv('SummaryStats/ErrorVSAngleLarge.csv')
    dfMain2.to_csv('SummaryStats/ErrorVSAngle.csv')

else: 
    raise ValueError('Did not process the data due to the processData flag being False. Make sure you want to process the data before proceeding!')

In [ ]:
Zs = np.sort(Z)
np.savetxt(testPath+geomPath+'ValErrorSort.csv', Zs, delimiter=',')

print('Mean Absolute Error (deg): ', str(np.mean(Zs)*180.0/math.pi))

# Choose what fraction of Zs to study
fracPred = 0.98


# Take the >fracPred set of best predictions
nKeep = int(np.ceil(fracPred*len(Zs)))
Zsmall = Zs[:nKeep]

# Print the fracPred quantile worst prediction
print(np.max(Zsmall))
print('This is equivalent to '+ str(np.max(Zsmall)*180.0/math.pi) + ' degrees')
print('Mean Absolute Error, best 98% (deg): ', str(np.mean(Zsmall)*180.0/math.pi))

plt.figure(2)
plt.hist(Zsmall, cumulative=False, density=True, bins=20)
plt.show()

zs = int(len(Zsmall)*0.84)
print(zs)
print(Zsmall.shape)
print(Zsmall[zs]*180/math.pi)

NN = np.array([1, 2, 5])
print(NN[0])
print(NN.shape)
GEOMVAL = np.arange(3, 7)
print(GEOMVAL)
print(GEOMVAL.shape)
# training_data, testing_data, epochs, fullAnglesVal, crosswireVal, trainPath, testPath = makeDataset(dataSetType=2, geometryVal=3, compFlag=True, N=1)

